In [ ]:
%pylab inline

In [ ]:
import pandas as pd
from IPython.display import display
import re

In [ ]:
counts = pd.read_table('read_counts.txt', index_col=0).T

rna_types = [s.split('.')[0] for s in counts.index.tolist() if s.endswith('.mapped')]
rna_types = 'univec,spikein,rRNA,miRNA,piRNA,Y_RNA,srpRNA,tRNA,snRNA,snoRNA,lncRNA,mRNA,tucpRNA'.split(',')
nongene_types = ['intron', 'promoter', 'enhancer', 'repeats']

def gradient_func(val):
    return '<span style="background: linear-gradient(90deg, #d65f5f {0}%, transparent 0%)">{0:.3f}</span>'.format(val)

columns = ['clean.unmapped']
rna_types_enabled = []
for rna_type in rna_types + nongene_types:
    if rna_type + '.mapped' in counts.columns:
        columns.append(rna_type + '.mapped')
        rna_types_enabled.append(rna_type)
rna_types = rna_types_enabled
columns += ['other.mapped']
counts_unmapped = counts.loc[:, 'other.unmapped']
counts = counts.loc[:, columns]
counts.rename(columns={key:re.sub('(\.mapped)|(\.unmapped)$', '', key) for key in columns}, inplace=True)
counts['unannotated'] = counts['other'].subtract(counts.loc[:, nongene_types].sum(axis=1), axis=0)
counts['unmapped'] = counts_unmapped
#counts['unmapped'] = counts['clean'].subtract(counts.loc[:, rna_types + ['other']].sum(axis=1), axis=0)
counts = counts.loc[:, ['clean'] + rna_types_enabled + ['unannotated', 'unmapped']]
display(counts.style \
        .set_caption('Read counts'))

In [ ]:
percent_by_clean = counts.copy()
percent_by_clean = 100*percent_by_clean.div(percent_by_clean.loc[:, 'clean'], axis=0)
percent_by_clean.drop(columns='clean', inplace=True)
display(percent_by_clean.style \
            .set_caption('Reads percentage by clean reads') \
            .format(gradient_func))

In [ ]:
display(percent_by_clean.mean(axis=0).to_frame().T.style \
            .set_caption('Average reads percentage by clean reads') \
            .format(gradient_func))

In [ ]:
percent_by_mapped = counts.copy()
rna_types_remove = [rna_type for rna_type in ['univec', 'rRNA', 'spikein', 'unmapped'] if rna_type in percent_by_mapped.columns]
percent_by_mapped = 100*percent_by_mapped.div(percent_by_mapped['clean'] \
                                              - percent_by_mapped.loc[:, rna_types_remove].sum(axis=1), axis=0)
percent_by_mapped.drop(columns=['clean'] + rna_types_remove, inplace=True)
display(percent_by_mapped.style \
            .set_caption('Reads percentage by mapped') \
            .format(gradient_func))

In [ ]:
display(percent_by_mapped.mean(axis=0).to_frame().T.style \
            .set_caption('Averge reads percentage by mapped reads') \
            .format(gradient_func))